In [1]:
import pandas as pd
import numpy as np
from pybaseball.statcast import statcast

In [2]:
statcast = statcast('2016-03-31', '2023-11-07')

This is a large query, it may take a moment to complete


/opt/homebrew/lib/python3.11/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates


100%|██████████| 1694/1694 [18:11<00:00,  1.55it/s]


In [3]:
# just making sure it's ordered
statcast.sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'], inplace=True)

In [4]:
# making a variable to tell me if a pitch is the last pitch of the inning so I know what the final score of the inning is
next_inning = statcast['inning'].shift(-1).fillna(0)
next_inning_topbot = statcast['inning_topbot'].shift(-1).fillna('')

statcast['last_of_inning'] = np.where((statcast['inning'] != next_inning) | 
                                      (statcast['inning_topbot'] != next_inning_topbot), 1, 0)

In [5]:
# getting the final score of the inning to compare with previous scores
statcast = statcast.reset_index(drop=True)

temp_df = statcast[statcast['last_of_inning'] == 1][['post_home_score', 'post_away_score']]

temp_df = temp_df.reindex(statcast.index).bfill()

statcast['end_of_inning_home_score'] = temp_df.post_home_score
statcast['end_of_inning_away_score'] = temp_df.post_away_score

In [6]:
# calculating difference in score from the pitch to the end of the inning
statcast['score_difference'] = statcast.end_of_inning_away_score - statcast.away_score + statcast.end_of_inning_home_score - statcast.home_score

In [8]:
# making features for base states, and seeing when the base-out state changes so I only use those pitches
statcast.on_1b = np.where(statcast.on_1b.isna(), 0, 1)
statcast.on_2b = np.where(statcast.on_2b.isna(), 0, 1)
statcast.on_3b = np.where(statcast.on_3b.isna(), 0, 1)

prev_on_3b = statcast['on_3b'].shift(1)
prev_on_2b = statcast['on_2b'].shift(1)
prev_on_1b = statcast['on_1b'].shift(1)
prev_outs = statcast['outs_when_up'].shift(1)

base_change = (statcast['on_3b'] != prev_on_3b) | \
              (statcast['on_2b'] != prev_on_2b) | \
              (statcast['on_1b'] != prev_on_1b) | \
              (statcast['outs_when_up'] != prev_outs)

statcast['base_out_change'] = base_change.astype(int)

In [9]:
# if the pitch did not result in a base-out change, the nullify
statcast.score_difference = np.where(statcast.base_out_change == 1, statcast.score_difference, np.nan)

In [19]:
# calculate RE24 by finding the average difference of score of each combination from the pitch to the end of the inning
re_24_groups = statcast.groupby(['outs_when_up', 'on_3b', 'on_2b', 'on_1b']).score_difference.mean().reset_index()

re_24_groups.rename(columns={'score_difference': 'run_exp', 'outs_when_up': 'outs'}, inplace=True)
re_24_groups.on_1b = np.where(re_24_groups.on_1b == 1, '1', '_')
re_24_groups.on_2b = np.where(re_24_groups.on_2b == 1, '2', '_')
re_24_groups.on_3b = np.where(re_24_groups.on_3b == 1, '3', '_')
re_24_groups['base_state'] = re_24_groups.on_1b + re_24_groups.on_2b + re_24_groups.on_3b
re_24_groups

,outs,on_3b,on_2b,on_1b,run_exp,base_state
0,0,_,_,_,0.51063,___
1,0,_,_,1,0.904437,1__
2,0,_,2,_,1.139816,_2_
3,0,_,2,1,1.495048,12_
4,0,3,_,_,1.391585,__3
5,0,3,_,1,1.765754,1_3
6,0,3,2,_,1.996903,_23
7,0,3,2,1,2.302265,123
8,1,_,_,_,0.273265,___
9,1,_,_,1,0.536009,1__


In [33]:
# reorganizing the table to easier viewing
re_24_table = re_24_groups.pivot_table(index='base_state', columns='outs', values='run_exp', aggfunc='first')

base_state_order = ['___', '1__', '_2_', '__3', '12_', '1_3', '_23', '123']
re_24_table.index = pd.CategoricalIndex(re_24_table.index, categories=base_state_order, ordered=True)
re_24_table = re_24_table.sort_index().astype(float)

re_24_table.round(3)

outs,0,1,2
base_state,,,
___,0.511,0.273,0.105
1__,0.904,0.536,0.230
_2_,1.140,0.699,0.325
__3,1.392,0.973,0.378
12_,1.495,0.948,0.455
1_3,1.766,1.197,0.514
_23,1.997,1.411,0.570
123,2.302,1.595,0.776
